In [1]:
import pandas as pd
import numpy as np
from scipy import spatial
from geopy.distance import vincenty

In [4]:
#data = pd.read_csv('columns_for_subway_merge.csv',index_col=0)
data = pd.read_csv('/Users/Laura/GoogleDrive/NYU/Capstone/NYCOpenData/Data/bronx_brooklyn_manhattan_queens_statenisland_2003_2016_bbls.csv')
data['bbl'] = data['bbl'].astype(int)
#data['zipcode'] = data['zipcode'].astype(int)
data = data.drop_duplicates().reset_index(drop=True)
data.head()

,bbl,latitude,longitude
0,1000010010,40.688799,-74.018675
1,1000047501,40.702153,-74.012021
2,1000050010,40.702776,-74.010722
3,1000057501,40.702266,-74.010607
4,1000070027,40.703290,-74.011022


In [9]:
def bbl_dist_to_subway(data):
    # Subway Enterance Locations
    subway = pd.read_csv('DOITT_SUBWAY_ENTRANCE_01_13SEPT2010.csv')
    subway = subway.drop('NAME',1)
    subway = subway.drop('URL',1)
    subway = subway.drop('LINE',1)
    subway['latitude'] = ''
    subway['longitude'] = ''
    for i in subway.index:
        temp = str(subway['the_geom'][i])
        temp = str.strip(temp, 'POINT (')
        temp = str.strip(temp, ')')
        temp = temp.split()
        subway['latitude'][i] = temp[1]
        subway['longitude'][i] = temp[0]
    subway = subway.drop('the_geom',1)
    subway = subway[['latitude', 'longitude']]
    subway = subway.convert_objects(convert_numeric=True)
    subway.head()

    subway_loc_matrix = subway.values.tolist()
    
    dist_list = []
    for i in data.index:
        if (i % 1000==0):
            print (i,)
        try:
            bbl_coord = (data['latitude'][i],data['longitude'][i])
            subway_coord = subway_loc_matrix[spatial.KDTree(subway_loc_matrix).query(bbl_coord)[1]]
            distance = vincenty(bbl_coord, subway_coord).miles
        except: 
            distance = np.nan
        dist_list = dist_list + [distance]
    data['subwaydist'] = dist_list 

In [10]:
data.shape

(316049, 3)

In [11]:
bbl_dist_to_subway(data)

/Users/Laura/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:18: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [14]:
data.head()

,bbl,latitude,longitude,zipcode,subwaydist
0,1000010010,40.688799,-74.018675,10004,0.927141
1,1000057501,40.702266,-74.010607,10004,0.106458
2,1000070038,40.703440,-74.011087,10004,0.111062
3,1000110012,40.704815,-74.012122,10004,0.066451
4,1000110014,40.704813,-74.011989,10004,0.070280


In [12]:
data.to_csv('subwaydist.csv',index=False)

In [ ]:
<merged_data>.merge(subwaydist,how='left')